## Test AWA Environment/Interface Plugins

In [1]:
from plugins.environments.awa_environment import AWAEnvironment
from plugins.interfaces.test_awa_interface import TestAWAInterface

env = AWAEnvironment(interface=TestAWAInterface(), variables={"Q6:BCTRL":[-1,1]})

In [2]:
env.dict()

{'name': 'awa_environment',
 'interface': {'name': 'awa_interface', 'params': {}, 'x': None, 'y': None},
 'variables': {'Q6:BCTRL': [-1.0, 1.0]},
 'observables': ['AWAICTMon:Ch1',
  'AWAICTMon:Ch2',
  'AWAICTMon:Ch3',
  'AWAICTMon:Ch4',
  'YAG1:XRMS',
  'YAG1:YRMS'],
 'params': {},
 'target_charge_PV': 'AWAICTMon:Ch1',
 'target_charge': None,
 'fractional_charge_deviation': 0.1}

In [3]:
env.set_variables({"Q6:BCTRL": 0.5})

In [4]:
env.get_observables(["YAG1:XRMS","YAG1:YRMS"])

{'YAG1:XRMS': 0.5, 'YAG1:YRMS': 0.0, 'AWAICTMon:Ch1': 1.0}

## Test with Xopt BE

In [5]:
from xopt import Xopt, Evaluator, VOCS
from xopt.generators.bayesian import BayesianExplorationGenerator

def evaluate(inputs):
    env.set_variables(inputs)
    return env.get_observables(env.observables)

vocs = VOCS(variables=env.variables, objectives={"YAG1:XRMS":"MINIMIZE"}, constraints={"YAG1:YRMS":["GREATER_THAN",0.0]})
generator = BayesianExplorationGenerator(vocs)
evaluator = Evaluator(function=evaluate)
X = Xopt(vocs=vocs, evaluator=evaluator, generator=generator)

In [6]:
vocs.dict()

{'variables': {'Q6:BCTRL': [-1.0, 1.0]},
 'constraints': {'YAG1:YRMS': ['GREATER_THAN', 0.0]},
 'objectives': {'YAG1:XRMS': 'MINIMIZE'},
 'constants': {},
 'linked_variables': {}}

In [9]:
import pandas as pd
X.evaluate_data(pd.DataFrame({"Q6:BCTRL":[0.0,0.1]}))

,Q6:BCTRL,AWAICTMon:Ch1,YAG1:XRMS,YAG1:YRMS,xopt_runtime,xopt_error
1,0.0,1.0,0.0,0.5,0.000075,False
2,0.1,1.0,0.1,0.4,0.000020,False


In [10]:
X.data

,Q6:BCTRL,AWAICTMon:Ch1,YAG1:XRMS,YAG1:YRMS,xopt_runtime,xopt_error
1,0.0,1.0,0.0,0.5,0.000075,False
2,0.1,1.0,0.1,0.4,0.000020,False


In [ ]:
for i in range(5):
    X.step()

In [ ]:
X.data

In [ ]:
X.data.plot(x="Q6:BCTRL", y=["YAG1:XRMS","YAG1:YRMS"],style="o")